In [ ]:
import pandas as pd
import numpy as np
import pickle

import plotly.express as px

### Read in the Service Fault Codes detailed info and do some EDA

In [ ]:
codes = pd.read_excel('../data/ServiceFaultCodes.xlsx')

In [ ]:
codes

In [ ]:
codes.columns

In [ ]:
# look at codes relating to exhaust
codes.loc[codes['Cummins Description'].str.contains('Exhaust')]

# there are 915(!) of them

In [ ]:
# look at codes relating to fuel
codes.loc[codes['Cummins Description'].str.contains('Fuel')]

# there are 574 of them

In [ ]:
# look at codes relating to fuel filter
codes.loc[codes['Cummins Description'].str.contains('Fuel Filter')]

# there are 15 of them

### Read in the pickle file of combined and filtered data and so some EDA

In [ ]:
on_faults = pd.read_pickle('../data/on_faults.pkl')

In [ ]:
on_faults.info()

#### Looking at fault events over time

In [ ]:
# Plot total faults by date
df = on_faults.groupby('EventDate').agg(total_faults = ('RecordID', 'count')).reset_index()


fig = px.scatter(df, x='EventDate', y='total_faults')
fig.show()

In [ ]:
# Plot unique trucks having issues by date
df = on_faults.groupby('EventDate').agg(total_faults = ('EquipmentID', 'nunique')).reset_index()


fig = px.scatter(df, x='EventDate', y='total_faults')
fig.show()

#### Now doing some aggregations by fault type

In [ ]:
on_faults.columns

In [ ]:
fault_overview = (
    on_faults
    .groupby(['spn', 'fmi'])
    .agg(total_occurrences = ('RecordID', 'count'),
         unique_trucks = ('EquipmentID', 'nunique'),
         min_date = ('EventDate', 'min'),
         max_date = ('EventDate', 'max'),
         unique_dates = ('EventDate', 'nunique'))
    .reset_index()
)


fault_overview['timespan'] = fault_overview['max_date'] -fault_overview['min_date']

fault_overview[['total_occurrences', 'unique_trucks', 
                'min_date', 'max_date', 
                'unique_dates', 'timespan']].describe(datetime_is_numeric=True)

In [ ]:
fault_overview.sort_values('total_occurrences', ascending = False).head(10)

In [ ]:
on_faults.loc[((on_faults['spn'] == 929)
             &
              (on_faults['fmi'] == 9))
             |
              ((on_faults['spn'] == 111)
             &
              (on_faults['fmi'] == 17))]

# There are 2 faults that account for nearly half of the data set

In [ ]:
# Removing the above faults since they are likely just to cause a lot of "noise" in the analysis
final_faults = on_faults.loc[~(((on_faults['spn'] == 929)
                              &
                              (on_faults['fmi'] == 9))
                             |
                             ((on_faults['spn'] == 111)
                              &
                              (on_faults['fmi'] == 17)))].reset_index(drop = True)

In [ ]:
final_faults.info()

In [ ]:
# Redo faults overview
fault_overview = (
    final_faults
    .groupby(['spn', 'fmi'])
    .agg(total_occurrences = ('RecordID', 'count'),
         unique_trucks = ('EquipmentID', 'nunique'),
         min_date = ('EventDate', 'min'),
         max_date = ('EventDate', 'max'),
         unique_dates = ('EventDate', 'nunique'))
    .reset_index()
)


fault_overview['timespan'] = fault_overview['max_date'] -fault_overview['min_date']

fault_overview[['total_occurrences', 'unique_trucks', 
                'min_date', 'max_date', 
                'unique_dates', 'timespan']].describe(datetime_is_numeric=True)

#### Doing some aggregations by truck ID

In [ ]:
truck_overview = (
    final_faults
    .groupby('EquipmentID')
    .agg(total_faults = ('RecordID', 'count'),
         unique_faults = ('spn', 'nunique'),
         min_date = ('EventDate', 'min'),
         max_date = ('EventDate', 'max'),
         unique_dates = ('EventDate', 'nunique'))
    .reset_index()
)


truck_overview['timespan'] = truck_overview['max_date'] - truck_overview['min_date']

truck_overview.describe(datetime_is_numeric=True)

In [ ]:
df = truck_overview
fig = px.histogram(df, 
                   x="total_faults",
                   marginal="box",
                   hover_data=df.columns)
fig.show()

In [ ]:
df = truck_overview
fig = px.histogram(df, 
                   x="unique_faults",
                   marginal="box",
                   hover_data=df.columns).update_traces(marker=dict(color='red'))
fig.show()

In [ ]:
df = truck_overview
fig = px.histogram(df, 
                   x="unique_dates",
                   marginal="box",
                   hover_data=df.columns).update_traces(marker=dict(color='green'))
fig.show()

In [ ]:
df = truck_overview
fig = px.histogram(df, 
                   x="timespan",
                   marginal="box",
                   hover_data=df.columns).update_traces(marker=dict(color='yellow'))
fig.show()

### Taking a look at full derates

In [ ]:
# Filter down to only include full derates
full_derates = final_faults.loc[final_faults['spn'] == 5246]



# Filter out faults where the light is going off
full_derates = full_derates.loc[full_derates['active'] == True].reset_index(drop = True)


full_derates

There are 493 instances of full rates that fit all the filtering criteria

In [ ]:
# Lets look only at the first full derate for a given truck on a given day
full_derates = full_derates.sort_values(['EquipmentID', 'EventTimeStamp'])

full_derates.loc[~full_derates.duplicated(['EquipmentID', 'EventDate'], keep = 'first')][['EquipmentID', 'EventDate']]

Dropping instances of repeated full derates for a given truck in a given day, we drop down to 383 occurrences

In [ ]:
# Taking a look at each truck that has had a full derate, how many unique timestamps each is associated with
truck_fd = (
    full_derates
    .groupby('EquipmentID')
    ['EventTimeStamp'].nunique()
    .to_frame()
    .reset_index()
    .rename(columns = {'EventTimeStamp' : 'date_count'})
    .sort_values('date_count', ascending = False)
    #.head(10)
)

truck_fd

194 trucks experienced full derates, 8 of those trucks experienced 10 or more derates

In [ ]:
# Taking a look at the timestamps for the truck with the largest number of unique timestamps
full_derates.loc[full_derates['EquipmentID'] == 1524]#['EventTimeStamp'].unique()

# It appears that this truck had multiple instances of multiple derates on the same day as well as derates separated
# by just a few days to a few weeks, all 32 derates occurred in a span of right about 6 months

In [ ]:
df = (
    full_derates
    .groupby('EquipmentID')
    ['EventDate'].nunique()
    .to_frame()
    .reset_index()
    .rename(columns = {'EventDate' : 'unique_dates'})
    .sort_values('unique_dates', ascending = False)
)

fig = px.histogram(df, 
                   x="unique_dates",
                   marginal="box",
                   hover_data=df.columns).update_traces(marker=dict(color='green'))
fig.show()

### Now look at partial derates

In [ ]:
# Filter down to only include full derates
partial_derates = final_faults.loc[(final_faults['spn'] == 1569)
                                &
                                (final_faults['fmi'] == 31)]



# Filter out faults where the light is going off
partial_derates = partial_derates.loc[partial_derates['active'] == True].reset_index(drop = True)


partial_derates

There are 5,062 instances that fit all filtering criteria

In [ ]:
truck_pd = (
    partial_derates
    .groupby('EquipmentID')
    ['EventTimeStamp'].nunique()
    .to_frame()
    .reset_index()
    .rename(columns = {'EventTimeStamp' : 'date_count'})
    .sort_values('date_count', ascending = False)
    #.head(42)
)

truck_pd

There are 548 trucks that experienced partial derates, 41 of which experienced 25 or more

In [ ]:
df = (
    partial_derates
    .groupby('EquipmentID')
    ['EventDate'].nunique()
    .to_frame()
    .reset_index()
    .rename(columns = {'EventDate' : 'unique_dates'})
    .sort_values('unique_dates', ascending = False)
)

fig = px.histogram(df, 
                   x="unique_dates",
                   marginal="box",
                   hover_data=df.columns).update_traces(marker=dict(color='green'))
fig.show()

In [ ]:
# How many trucks that experienced full derates also experienced partial derates?
truck_bd = truck_fd.loc[truck_fd['EquipmentID'].isin(truck_pd['EquipmentID'])]

len(truck_bd)
# 163 of the 194

In [ ]:
timing = (
    final_faults.loc[(final_faults['EquipmentID'].isin(truck_bd['EquipmentID']))
                     &
                     ((final_faults['spn'] == 5246)
                      |
                      ((final_faults['spn'] == 1569)
                       &
                       (final_faults['fmi'] == 31)))]
    .loc[final_faults['active'] == True]
    [['EquipmentID', 'EventDate','EventTime', 'spn', 'fmi']]
    .sort_values(by = ['EquipmentID', 'EventDate', 'EventTime'])
    .reset_index(drop = True)
)

timing

In [ ]:
timing = timing.loc[~timing.duplicated(['EquipmentID', 'EventDate', 'spn', 'fmi'], keep = 'first')].reset_index(drop = True)

In [ ]:
timing_sameday = timing.loc[timing.duplicated(['EquipmentID', 'EventDate'], keep=False)]

timing_sameday

In [ ]:
# When a partial derate and full derate occur in the same day, which occurs first?

sameday_firsts = timing_sameday.loc[timing_sameday.duplicated(['EquipmentID', 'EventDate'], keep = 'last')]

sameday_firsts['spn'].value_counts()

# This happens 178 times, partial derate occurs first 153 times

In [ ]:
timing_sameday['EventTime'] = [pd.to_timedelta(x) for x in timing_sameday['EventTime'].astype(str)]

In [ ]:
timing_sameday['dif'] = timing_sameday.groupby(['EquipmentID', 'EventDate'])['EventTime'].diff()

In [ ]:
weirds = timing_sameday.loc[~timing_sameday['dif'].isna()].loc[timing_sameday['spn'] == 1569].sort_values('dif').tail(20)

weirds

The timing difference between the first instance of a full derate or partial derate and the subsequent first occurrence of the other code on the same day can be as little as 0 time and just over 20 hours

In [ ]:
(final_faults.loc[(final_faults['EquipmentID'].isin(weirds['EquipmentID']))
                  & 
                 ((final_faults['spn'] == 5246)
                  |
                  ((final_faults['spn'] == 1569) 
                  &
                  (final_faults['fmi'] == 31)))
                 & (final_faults['EventDate'].isin(weirds['EventDate']))]
 .sort_values(['EquipmentID', 'EventTimeStamp'])
 [['EquipmentID', 'EventTimeStamp', 'spn', 'active']]
 .head(60)
)

### Now lets look at all the faults for the trucks with full derates

In [ ]:
final_faults['EquipmentID'].nunique()

In [ ]:
final_faults_fd = final_faults.loc[final_faults['EquipmentID'].isin(truck_fd['EquipmentID'])]

final_faults_fd
# ~23.5% of the filtered fault records are attributible to trucks that had full derates, 11.05% of all trucks in the df

In [ ]:
final_faults_fd_overview = (
    final_faults_fd
    .groupby(['spn', 'fmi'])
    .agg(total_occurrences = ('RecordID', 'count'),
         unique_trucks = ('EquipmentID', 'nunique'),
         min_date = ('EventDate', 'min'),
         max_date = ('EventDate', 'max'),
         unique_dates = ('EventDate', 'nunique'))
    .reset_index(drop=False)
    .sort_values('total_occurrences', ascending=False)
)

final_faults_fd_overview['proportion'] = final_faults_fd_overview['total_occurrences']/len(final_faults_fd)

final_faults_fd_overview.head(20)

In [ ]:
fault_overview['proportion'] = fault_overview['total_occurrences']/len(final_faults)

fault_overview.sort_values('total_occurrences', ascending=False).head(20)

In [ ]:
final_faults.loc[(final_faults['spn'] == 1068)
                 &
                 (final_faults['fmi'] == 2)]